<a href="https://colab.research.google.com/github/jd-velasquezr/TG_Fresas/blob/main/EntrenamientoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# CÓDIGO DE ENTRENAMIENTO, EXTRACCIÓN DE RESULTADOS Y GUARDADO DE LOS CLASIFICADORES.
# Realizado por: Juan D. Velásquez y Daniel Barandica.

# A continuación se escribe la completa importación de las librerías que se
# implementarán a lo largo del código.
import os
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd
!pip install XlsxWriter
import xlsxwriter
import warnings
warnings.filterwarnings('ignore')
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import KFold
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.preprocessing import RobustScaler
from sklearn.naive_bayes import GaussianNB
import time
import pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab.patches import cv2_imshow
# Ruta del Dataset de entrada.
path = '/content/drive/MyDrive/Img_Dataset/'
sets = os.listdir(path)

# Definición de Arreglos asociados a los datos para procesar.
train_X, train_Y = ([], [])

data = [(train_X, train_Y)]

# Ciclo por los directorios del Dataset.
for s, d in (zip(sets, data)):
    # Definción de las direcciones de las carpetas con dos categorías.
    path_to_cat = os.path.join(path, s)
    
    # Ciclado a través de las categorías en la carpeta del Dataset.
    for cat in os.listdir(path_to_cat):
        # Definiendo las direcciones a las imagénes en cada categoría.
        path_to_images = os.path.join(path_to_cat, cat)
        print(path_to_images)
        # Definición de Etiquetas
        if cat == 'Healthy':
            label = 0
        elif cat == 'Sick':
            label = 1
        else:
          break
        
        # Recorre cada imagen en la categoría (Whole o Flesh)
        for i in os.listdir(path_to_images):
            # Dirección a la imagen.
            image_path = os.path.join(path_to_images, i)
            # Lectura de la imagen en cuestión.
            img = cv.imread(image_path)
            # A continuación se almacenan los canales de la imagen en una
            # variable al igual que las variables asociadas a los canales
            # a iterar y el vector de características.
            chans = cv.split(img)
            colors = ("b", "g", "r")
            features = []

            # Se procesan los canales de la imagen, siguiendo un histograma
            # de 16 bins leyendo píxeles de valores de 0 a 255.
            for (chan, color) in zip(chans, colors):
              hist = cv.calcHist([chan], [0], None, [16], [0, 255])
              # Se colocan los valores del histograma en el arreglo features.
              features.extend(hist)
            
            # A continuación se almacenan las características y las respectivas
            # etiquetas de los datos, donde en primer medida se aplana el vector
            # features para que se obtenga un vector de una dimensión.
            features = np.array(features).flatten()
            d[0].append(features)
            d[1].append(label)

/content/drive/MyDrive/Img_Dataset/Full/Healthy
/content/drive/MyDrive/Img_Dataset/Full/Sick


In [ ]:
# Se convierten los arreglos de datos a vectores de tipo numpy.
data_X = np.array(train_X)
data_Y = np.array(train_Y)

# Se construye un dataframe para visualizar los datos de las imágenes con
# sus etiquetas.
dX = pd.DataFrame(data_X)
dY = pd.DataFrame(data_Y)
datos = pd.concat([dX, dY], axis = 1)

(2301, 48)
(2301,)


,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,0
0,43573.0,1140.0,1293.0,1234.0,794.0,374.0,175.0,84.0,45.0,68.0,...,158.0,73.0,70.0,66.0,44.0,45.0,49.0,71.0,138.0,0
1,46000.0,377.0,691.0,853.0,554.0,267.0,108.0,56.0,39.0,43.0,...,98.0,43.0,26.0,22.0,20.0,32.0,50.0,49.0,48.0,0
2,46532.0,429.0,508.0,545.0,396.0,224.0,98.0,56.0,48.0,32.0,...,54.0,31.0,29.0,36.0,50.0,65.0,59.0,66.0,93.0,0
3,44222.0,995.0,1230.0,1079.0,754.0,344.0,114.0,64.0,35.0,32.0,...,115.0,65.0,61.0,46.0,43.0,28.0,29.0,43.0,70.0,0
4,45956.0,597.0,755.0,673.0,551.0,222.0,92.0,53.0,40.0,28.0,...,63.0,27.0,37.0,30.0,47.0,29.0,36.0,52.0,71.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2296,48316.0,127.0,120.0,93.0,55.0,47.0,37.0,34.0,35.0,40.0,...,8.0,14.0,22.0,23.0,19.0,34.0,36.0,47.0,79.0,1
2297,48672.0,79.0,51.0,36.0,25.0,18.0,27.0,21.0,15.0,30.0,...,6.0,8.0,15.0,16.0,14.0,31.0,21.0,26.0,47.0,1
2298,46047.0,517.0,678.0,648.0,515.0,299.0,122.0,52.0,30.0,31.0,...,72.0,34.0,11.0,20.0,22.0,16.0,27.0,37.0,67.0,1
2299,48595.0,47.0,26.0,30.0,36.0,29.0,33.0,31.0,58.0,51.0,...,20.0,10.0,16.0,15.0,18.0,34.0,50.0,58.0,79.0,1


**SEPARACIÓN DE DATASET Y ESCALIZACIÓN**

In [ ]:
# Se separa el dataset en entrenamiento y validacion, partiendo de 80% entrenamiento y 20% validación.
X_train,X_test,Y_train,Y_test = train_test_split(data_X,data_Y,test_size=0.2, random_state = 1, shuffle = True)

# Se construye un y se aplica un escalizador robusto para asegurar que sea resistente frente a
# outliers de los datos de las imágenes y a su vez mejorar la convergencia de 
# de la sintonización de paramétros de los algoritmos de ML.
transformer = RobustScaler().fit(X_train)
X_train = transformer.transform(X_train)
X_test = transformer.transform(X_test)

(461,)


**SVM**

In [ ]:
# Variable que contienen todos los hiperparámetros del algoritmo, de los cuales
# se van a iterar.
parameters = {'kernel':('rbf','sigmoid','poly'), 'C':[1,40,50,60,70,80,90,95], 
              'decision_function_shape': ('ovo', 'ovr'),
              'gamma': [0.001,0.003,0.01,0.03,0.1,0.3,1,3,10,30,'auto']}
# Declaración de la SVM.
svc = svm.SVC()

# Esta parte realiza la búsqueda de gradiente a través de validación cruzada con
# los parámetros dados.
clf = GridSearchCV(svc, parameters, cv = 5, refit = 'True', verbose = 0, n_jobs = -1)

# Se efectúa el entrenamiento del modelo con los mejores parámetros.
clf.fit(X_train, Y_train)
sorted(clf.cv_results_.keys())

Mejores Hiperparámetros: {'C': 90, 'decision_function_shape': 'ovo', 'gamma': 0.01, 'kernel': 'rbf'}


In [ ]:
# En el siguiente proceso se crea una variable de tiempo tal que contiene todos
# los tiempos obtenidos del algoritmo para clasificar todo el conjunto de 
# validación en 100 intentos.
times = np.array([])
for i in range(1,100):
  start = time.time()
  y_test_predicted = clf.predict(X_test)
  stop = time.time()
  times = np.append(times, stop - start)

# Se realiza un promedio de los tiempos de los 100 eventos.
tc = np.average(times, axis = 0)

# A continuación se realizan las evaluaciones del modelo obteniendo el MCC
# y accuracy.
y_test_scores = clf.decision_function(X_test)
MCC = matthews_corrcoef(Y_test, y_test_predicted)
print("Resultado mediante MCC:", MCC)
ACC = accuracy_score(Y_test, y_test_predicted)
print("Resultado mediante Accuracy", ACC)
print("\n") 

# Impresión y guardado de un dataframe con los resultados del modelo.
report_dict = classification_report(Y_test, y_test_predicted, output_dict=True)
reporte = pd.DataFrame(report_dict)
reporte.drop(['accuracy'], axis=1, inplace = True)
reporte.drop(['support'], axis=0, inplace = True)
print("Reporte de resultados en cada clase:\n") 

report_PRES_SVM = pd.DataFrame(data = {'Parámetros': ["MCC", "ACC", "PROCC_TIME","HIPERP" ], 'Valores': [MCC, ACC, tc, str(clf.best_params_)]})
print(tc)



Resultado mediante MCC: 0.9563426296100571
Resultado mediante Accuracy 0.9804772234273319


Reporte de resultados en cada clase:



0.00905980004204644


**KNN**

In [ ]:
# Parámetros a iterar.
estimator_KNN = KNeighborsClassifier(algorithm='auto')
parameters_KNN = {
    'n_neighbors': (11,23,25,27,30,100),
    'leaf_size': (30,40),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski','chebyshev')}

# Validación cruzada del modelo.
grid_search_KNN = GridSearchCV(estimator_KNN, param_grid=parameters_KNN, n_jobs = -1, cv = 5)
# Entrenamiento del mejor modelo.
grid_search_KNN.fit(X_train, Y_train)

Training time: 4.017763137817383s
Mejores Hiperparámetros: {'leaf_size': 30, 'metric': 'minkowski', 'n_neighbors': 30, 'p': 2, 'weights': 'distance'}




In [ ]:
# Como se evidenció en el código de la SVM, se repite el proceso obteniendo
# el tiempo de clasificación en 100 eventos, luego se ejecuta la evaluación
# del modelo y finalmente el guardado de los resultados en un dataframe.
times = np.array([])
for i in range(1,100):
  start = time.time()
  y_test_predicted = grid_search_KNN.predict(X_test)
  stop = time.time()
  times = np.append(times, stop - start)

tc = np.average(times, axis = 0)
##
MCC = matthews_corrcoef(Y_test, y_test_predicted)
print("Resultado mediante MCC:", MCC)
ACC = accuracy_score(Y_test, y_test_predicted)
print("Resultado mediante Accuracy", ACC)
##
report_dict = classification_report(Y_test, y_test_predicted, output_dict=True)
reporte = pd.DataFrame(report_dict)
reporte.drop(['accuracy'], axis=1, inplace = True)
print("Reporte de resultados en cada clase:\n")
display(reporte) 
print("\n")

report_PRES_KNN = pd.DataFrame(data = {'Parámetros': ["MCC", "ACC", "PROCC_TIME","HIPERP" ], 'Valores': [MCC, ACC, tc, str(grid_search_KNN.best_params_)]})
print(tc)

Resultado mediante MCC: 0.9319271258519073
Resultado mediante Accuracy 0.9696312364425163


Reporte de resultados en cada clase:



,0,1,macro avg,weighted avg
precision,0.970874,0.967105,0.968990,0.969599
recall,0.983607,0.942308,0.962957,0.969631
f1-score,0.977199,0.954545,0.965872,0.969533
support,305.000000,156.000000,461.000000,461.000000




0.030079723608614217


**NAIVE BAYES GAUSSIANO**

In [ ]:
# Declaración y entrenamiento del clasificador.
clf_NB = GaussianNB()
clf_NB.fit(X_train, Y_train)

Training time: 0.013350725173950195s


In [ ]:
# Como se evidenció en el código de la SVM, se repite el proceso obteniendo
# el tiempo de clasificación en 100 eventos, luego se ejecuta la evaluación
# del modelo y finalmente el guardado de los resultados en un dataframe.
times = np.array([])
for i in range(1,100):
  start = time.time()
  y_test_predicted = clf_NB.predict(X_test)
  stop = time.time()
  times = np.append(times, stop - start)
tc = np.average(times, axis = 0)
##
MCC = matthews_corrcoef(Y_test, y_test_predicted)
print("Resultado mediante MCC:", MCC)
ACC = accuracy_score(Y_test, y_test_predicted)
print("Resultado mediante Accuracy", ACC)
##
report_dict = classification_report(Y_test, y_test_predicted, output_dict=True)
reporte = pd.DataFrame(report_dict)
reporte.drop(['accuracy'], axis=1, inplace = True)
print("Reporte de resultados en cada clase:\n")
display(reporte) 
print("\n")

report_PRES_NV = pd.DataFrame(data = {'Parámetros': ["MCC", "ACC", "PROCC_TIME" ], 'Valores': [MCC, ACC, tc]})

Resultado mediante MCC: 0.5171867085443416
Resultado mediante Accuracy 0.7245119305856833


Reporte de resultados en cada clase:



,0,1,macro avg,weighted avg
precision,0.936275,0.556420,0.746347,0.807734
recall,0.626230,0.916667,0.771448,0.724512
f1-score,0.750491,0.692494,0.721493,0.730865
support,305.000000,156.000000,461.000000,461.000000


**REGRESIÓN LOGÍSTICA**

In [ ]:
# Selección de los parámetros del modelo a iterar.
parameters = {'penalty':('l2','l1'), 'C':[0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10]}

# Declaración del modelo de clasificación.
log_reg=LogisticRegression()
logreg_cv=GridSearchCV(log_reg, parameters, cv= 5, refit = 'True', verbose = 0, scoring = 'accuracy')

# Entrenamiento del modelo con mejores resultados.
logreg_cv.fit(X_train, Y_train)

Training time: 0.34257984161376953s


['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_C',
 'param_penalty',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

**PRUEBA DEL MODELO**

In [ ]:
# Como se evidenció en el código de la SVM, se repite el proceso obteniendo
# el tiempo de clasificación en 100 eventos, luego se ejecuta la evaluación
# del modelo y finalmente el guardado de los resultados en un dataframe.NDO TÉCNICAS COMO EL ACCURACY, EL MCC Y F1 SCORE.
times = np.array([])
for i in range(1,100):
  start = time.time()
  y_test_predicted = logreg_cv.predict(X_test)
  stop = time.time()
  times = np.append(times, stop - start)
tc = np.average(times, axis = 0)
##
MCC = matthews_corrcoef(Y_test, y_test_predicted)
print("Resultado mediante MCC:", MCC)
ACC = accuracy_score(Y_test, y_test_predicted)
print("Resultado mediante Accuracy", ACC)
##
report_dict = classification_report(Y_test, y_test_predicted, output_dict=True)
reporte = pd.DataFrame(report_dict)
reporte.drop(['accuracy'], axis=1, inplace = True)
print("Reporte de resultados en cada clase:\n")
display(reporte) 
##
report_PRES_LR = pd.DataFrame(data = {'Parámetros': ["MCC", "ACC", "PROCC_TIME", "HIPERP" ], 'Valores': [MCC, ACC, tc, str(logreg_cv.best_params_)]})
print(tc)

Mejores Hiperparámetros: {'C': 0.0003, 'penalty': 'l2'}


Resultado mediante MCC: 0.6998514329618791
Resultado mediante Accuracy 0.8655097613882863
Reporte de resultados en cada clase:



,0,1,macro avg,weighted avg
precision,0.840336,0.951923,0.896130,0.878097
recall,0.983607,0.634615,0.809111,0.865510
f1-score,0.906344,0.761538,0.833941,0.857343
support,305.000000,156.000000,461.000000,461.000000




0.000407141868514244


**ANN DE 128 NEURONAS EN CAPA 2**

**ANN DE 64 NEURONAS EN CAPA 3**

**ANN DE 32 NEURONAS EN CAPA 4**

In [ ]:
# Como primera medida, se asegura que no haya ninguna sesión abierta de keras 
# que pueda interferar en los procesos.
tf.keras.backend.clear_session()
# Selección de número de folds en cross validación.
num_folds = 5

# Definición de los vectores que contienen la precisión y el loss de cada fold.
acc_per_fold = []
loss_per_fold = []

# Definición de entradas y etiquetas de los datos de entrenamiento.
inputs = X_train
targets = Y_train

# Definición de los parámetros del validador cruzado.
kfold = KFold(n_splits=num_folds, shuffle=False)

# Inicio de validación cruzada para el modelo a través de cinco ciclos en todos
# los datos.
fold_no = 1
for train, test in kfold.split(inputs, targets):

# Se declaran los hiperparámetros referentes a cada neurona.
  model128 = Sequential()
  model128.add(Dense(128, input_shape = (shape,), activation = 'relu', kernel_initializer = 'he_normal'))
  model128.add(Dense(64, activation = 'relu', kernel_initializer = 'he_normal'))
  model128.add(Dense(32, activation = 'relu', kernel_initializer = 'he_normal'))
  model128.add(Dense(16, activation = 'relu', kernel_initializer = 'he_normal'))
  model128.add(Dense(1, activation = 'sigmoid'))

  # Se define un punto de control tal que guarda el mejor modelo de cada fold.
  checkpoint = ModelCheckpoint(f'model_128_{fold_no}.hdf5', monitor = 'loss', verbose = 0, save_best_only = True)
  # Se escriben algunos parámetros adicionales en el proceso de compilación de
  # red neural.
  model128.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


  # Se  muestran los resultados.
  print('------------------------------------------------------------------------')
  print(f'Entrenando para el fold no. {fold_no} ...')

  # Inicio de entrenamiento del modelo con los respectivos parámetros.
  history = model128.fit(inputs[train], targets[train],
              batch_size = 256, epochs = 1000, callbacks = [checkpoint])

  # Se generan métricas de generalización de cada época.
  scores = model128.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model128.metrics_names[0]} of {scores[0]}; {model128.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Aumento del número del fold.
  fold_no = fold_no + 1
  
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

Se han truncado las últimas 5000 líneas del flujo de salida.
Epoch 513/1000
6/6 [==============================] - 0s 15ms/step - loss: 3.4827e-06 - accuracy: 1.0000
Epoch 514/1000
6/6 [==============================] - 0s 18ms/step - loss: 3.4608e-06 - accuracy: 1.0000
Epoch 515/1000
6/6 [==============================] - 0s 15ms/step - loss: 3.4436e-06 - accuracy: 1.0000
Epoch 516/1000
6/6 [==============================] - 0s 13ms/step - loss: 3.4233e-06 - accuracy: 1.0000
Epoch 517/1000
6/6 [==============================] - 0s 14ms/step - loss: 3.4056e-06 - accuracy: 1.0000
Epoch 518/1000
6/6 [==============================] - 0s 14ms/step - loss: 3.3912e-06 - accuracy: 1.0000
Epoch 519/1000
6/6 [==============================] - 0s 15ms/step - loss: 3.3697e-06 - accuracy: 1.0000
Epoch 520/1000
6/6 [==============================] - 0s 13ms/step - loss: 3.3515e-06 - accuracy: 1.0000
Epoch 521/1000
6/6 [==============================] - 0s 13ms/step - loss: 3.3337e-06 - accuracy: 1

**PRUEBAS DEL MODELO**

In [ ]:
# Se crea un arreglo tal que contiene los resultados del MCC de cada red neural
# entrenada de cada fold.
mc = np.array([])
for i in range(1,5):
  model = tf.keras.models.load_model(f'/content/model_128_{i}.hdf5')
  y_pred = np.around(np.array(model.predict(X_test)))
  MCC = matthews_corrcoef(Y_test, y_pred)
  mc = np.append(mc,MCC)

# Esta línea obtiene la mejor red y la importa.
model = tf.keras.models.load_model(f'/content/model_128_{np.argmax(mc)+1}.hdf5')

# la sección posterior realiza la evaluación en tiempo del clasificador y
# almacena los resultados en un vector, luego efectúa el promedio.
times = np.array([])
for i in range(1,10):
  start = time.perf_counter_ns()
  y_pred = model.predict(X_test)
  stop = time.perf_counter_ns()
  times = np.append(times, stop - start)

tc = np.average(times, axis = 0)/(10**9)
y_pred = np.around(np.array(y_pred))

# En esta parte se ejecuta la evaluación del modelo en MCC y accuracy.
MCC = matthews_corrcoef(Y_test, y_pred)
print("Resultado mediante MCC:", MCC)
ACC = accuracy_score(Y_test, y_pred)
print("Resultado mediante Accuracy", ACC)
print("\n")
print("El mejor fue: ", np.argmax(mc)+1)

# Se guardan los resultados en tiempo, accuracy y MCC en un dataframe.
REPORT_PRES_ANN128 = pd.DataFrame(data = {'Parámetros': ["MCC", "ACC", "PROCC_TIME" ], 'Valores': [MCC, ACC, tc]})
display(REPORT_PRES_ANN128)

15/15 [==============================] - 0s 2ms/step
Resultado mediante MCC: 0.9708877136918846
Resultado mediante Accuracy 0.9869848156182213


El mejor fue:  2


,Parámetros,Valores
0,MCC,0.970888
1,ACC,0.986985
2,PROCC_TIME,0.117223


****ANN DE 64 NEURONAS EN CAPA 1**

**ANN DE 32 NEURONAS EN CAPA 2**

**ANN DE 16 NEURONAS EN CAPA 3****

In [ ]:
# Como se observó en el entrenamiento de la ANN de 128 neuronas, se repite el
# mismo proceso previo, pero ahora aplicado a una ANN de 64 a 8 neuronas
# manteniendo los mismos parámetros de cada línea a excepción del número de
# de neuronas.
tf.keras.backend.clear_session()
num_folds = 5

acc_per_fold = []
loss_per_fold = []

inputs = X_train
targets = Y_train

kfold = KFold(n_splits=num_folds, shuffle=True)
fold_no = 1
for train, test in kfold.split(inputs, targets):

  model64 = Sequential()
  model64.add(Dense(64, input_shape = (shape,), activation = 'relu', kernel_initializer = 'he_normal'))
  model64.add(Dense(32, activation = 'relu', kernel_initializer = 'he_normal'))
  model64.add(Dense(16, activation = 'relu', kernel_initializer = 'he_normal'))
  model64.add(Dense(8, activation = 'relu', kernel_initializer = 'he_normal'))
  model64.add(Dense(1, activation = 'sigmoid'))

  checkpoint = ModelCheckpoint(f'model64_{fold_no}.hdf5', monitor = 'loss', verbose = 0, save_best_only = True)

  model64.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  history = model64.fit(inputs[train], targets[train],
              batch_size = 256, epochs = 1000, callbacks = [checkpoint])
  
  scores = model64.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model64.metrics_names[0]} of {scores[0]}; {model64.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  fold_no = fold_no + 1

print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

Se han truncado las últimas 5000 líneas del flujo de salida.
Epoch 513/1000
6/6 [==============================] - 0s 11ms/step - loss: 1.9282e-05 - accuracy: 1.0000
Epoch 514/1000
6/6 [==============================] - 0s 11ms/step - loss: 1.9155e-05 - accuracy: 1.0000
Epoch 515/1000
6/6 [==============================] - 0s 12ms/step - loss: 1.9062e-05 - accuracy: 1.0000
Epoch 516/1000
6/6 [==============================] - 0s 14ms/step - loss: 1.8955e-05 - accuracy: 1.0000
Epoch 517/1000
6/6 [==============================] - 0s 12ms/step - loss: 1.8849e-05 - accuracy: 1.0000
Epoch 518/1000
6/6 [==============================] - 0s 13ms/step - loss: 1.8724e-05 - accuracy: 1.0000
Epoch 519/1000
6/6 [==============================] - 0s 12ms/step - loss: 1.8631e-05 - accuracy: 1.0000
Epoch 520/1000
6/6 [==============================] - 0s 12ms/step - loss: 1.8518e-05 - accuracy: 1.0000
Epoch 521/1000
6/6 [==============================] - 0s 12ms/step - loss: 1.8418e-05 - accuracy: 1

**PRUEBAS DEL MODELO**

In [ ]:
# Nuevamente, se repite el proceso visto en la evaluación de la
# ANN de 128 neuronas, manteniendo las métricas de tiempos, MCC y accuracy al
# igual que el guardado de los resultados en un dataframe.
mc = np.array([])
for i in range(1,5):
  model = tf.keras.models.load_model(f'/content/model64_{i}.hdf5')
  y_pred = np.around(np.array(model.predict(X_test)))
  MCC = matthews_corrcoef(Y_test, y_pred)
  mc = np.append(mc,MCC)

model = tf.keras.models.load_model(f'/content/model64_{np.argmax(mc)+1}.hdf5')

times = np.array([])
for i in range(1,10):
  start = time.perf_counter_ns()
  y_pred = np.around(np.array(model.predict(X_test)))
  stop = time.perf_counter_ns()
  times = np.append(times, stop - start)

y_pred = np.around(np.array(model.predict(X_test)))
tc = np.average(times, axis = 0)/(10**9)

MCC = matthews_corrcoef(Y_test, y_pred)
print("Resultado mediante MCC:", MCC)
ACC = accuracy_score(Y_test, y_pred)
print("Resultado mediante Accuracy", ACC)
print("\n")
print("El mejor fue: ", np.argmax(mc)+1)

REPORT_PRES_ANN64 = pd.DataFrame(data = {'Parámetros': ["MCC", "ACC", "PROCC_TIME" ], 'Valores': [MCC, ACC, tc]})
display(REPORT_PRES_ANN64)

15/15 [==============================] - 0s 2ms/step
Resultado mediante MCC: 0.9660468833630064
Resultado mediante Accuracy 0.9848156182212582


El mejor fue:  1


,Parámetros,Valores
0,MCC,0.966047
1,ACC,0.984816
2,PROCC_TIME,0.105407


**ANN DE 32 NEURONAS EN CAPA 1**

**ANN DE 16 NEURONAS EN CAPA 2**

**ANN DE 8 NEURONAS EN CAPA 3**

In [ ]:
# Como se observó en el entrenamiento de la ANN de 128 neuronas, se repite el
# mismo proceso previo, pero ahora aplicado a una ANN de 32 a 4 neuronas
# manteniendo los mismos parámetros de cada línea a excepción del número de
# de neuronas.
tf.keras.backend.clear_session()
num_folds = 5

acc_per_fold = []
loss_per_fold = []

inputs = X_train
targets = Y_train

kfold = KFold(n_splits=num_folds, shuffle=True)

fold_no = 1
for train, test in kfold.split(inputs, targets):

  model32 = Sequential()
  model32.add(Dense(32, input_shape = (shape,), activation = 'relu', kernel_initializer = 'he_normal'))
  model32.add(Dense(16, activation = 'relu', kernel_initializer = 'he_normal'))
  model32.add(Dense(8, activation = 'relu', kernel_initializer = 'he_normal'))
  model32.add(Dense(4, activation = 'relu', kernel_initializer = 'he_normal'))
  model32.add(Dense(1, activation = 'sigmoid'))

  checkpoint = ModelCheckpoint(f'model32_{fold_no}.hdf5', monitor = 'loss', verbose = 0, save_best_only = True)

  model32.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  history = model32.fit(inputs[train], targets[train],
              batch_size = 256, epochs = 1000, callbacks = [checkpoint])

  scores = model32.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model32.metrics_names[0]} of {scores[0]}; {model32.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  fold_no = fold_no + 1

print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

Se han truncado las últimas 5000 líneas del flujo de salida.
Epoch 513/1000
6/6 [==============================] - 0s 11ms/step - loss: 2.4821e-05 - accuracy: 1.0000
Epoch 514/1000
6/6 [==============================] - 0s 13ms/step - loss: 2.4678e-05 - accuracy: 1.0000
Epoch 515/1000
6/6 [==============================] - 0s 12ms/step - loss: 2.4512e-05 - accuracy: 1.0000
Epoch 516/1000
6/6 [==============================] - 0s 11ms/step - loss: 2.4429e-05 - accuracy: 1.0000
Epoch 517/1000
6/6 [==============================] - 0s 12ms/step - loss: 2.4239e-05 - accuracy: 1.0000
Epoch 518/1000
6/6 [==============================] - 0s 11ms/step - loss: 2.4117e-05 - accuracy: 1.0000
Epoch 519/1000
6/6 [==============================] - 0s 15ms/step - loss: 2.3957e-05 - accuracy: 1.0000
Epoch 520/1000
6/6 [==============================] - 0s 12ms/step - loss: 2.3861e-05 - accuracy: 1.0000
Epoch 521/1000
6/6 [==============================] - 0s 11ms/step - loss: 2.3701e-05 - accuracy: 1

**PRUEBAS DEL MODELO**

In [ ]:
# Nuevamente, se repite el proceso visto en la evaluación de la
# ANN de 128 neuronas, manteniendo las métricas de tiempos, MCC y accuracy al
# igual que el guardado de los resultados en un dataframe.
mc = np.array([])
for i in range(1,5):
  model = tf.keras.models.load_model(f'/content/model32_{i}.hdf5')
  y_pred = np.around(np.array(model.predict(X_test)))
  MCC = matthews_corrcoef(Y_test, y_pred)
  mc = np.append(mc,MCC)

model = tf.keras.models.load_model(f'/content/model32_{np.argmax(mc)+1}.hdf5')

times = np.array([])
for i in range(1,10):
  start = time.perf_counter_ns()
  y_pred = np.around(np.array(model.predict(X_test)))
  stop = time.perf_counter_ns()
  times = np.append(times, stop - start)

y_pred = np.around(np.array(model.predict(X_test)))
tc = np.average(times, axis = 0)/(10**9)

MCC = matthews_corrcoef(Y_test, y_pred)
print("Resultado mediante MCC:", MCC)
ACC = accuracy_score(Y_test, y_pred)
print("Resultado mediante Accuracy", ACC)
print("\n")
print("El mejor fue: ", np.argmax(mc)+1)

REPORT_PRES_ANN32 = pd.DataFrame(data = {'Parámetros': ["MCC", "ACC", "PROCC_TIME" ], 'Valores': [MCC, ACC, tc]})
display(REPORT_PRES_ANN32)

15/15 [==============================] - 0s 2ms/step
Resultado mediante MCC: 0.9564790984037256
Resultado mediante Accuracy 0.9804772234273319


El mejor fue:  3


,Parámetros,Valores
0,MCC,0.956479
1,ACC,0.980477
2,PROCC_TIME,0.101895


**ESCRITURA DE LOS RESULTADOS DE LOS CLASIFICADORES**

In [ ]:
# Se declara una variable que contiene los parámetros de los cuales
# se va a guardar la información de los resultados de los algoritmos
# de clasificación.
Cf_Results = pd.ExcelWriter('CfResults.xlsx', engine='xlsxwriter')

# Se escribe cada dataframe a una hoja diferente del archivo de excel.
report_PRES_SVM.to_excel(Cf_Results, sheet_name='SVM')
report_PRES_KNN.to_excel(Cf_Results, sheet_name='KNN')
report_PRES_LR.to_excel(Cf_Results, sheet_name='LR')
report_PRES_NV.to_excel(Cf_Results, sheet_name='NV')
REPORT_PRES_ANN128.to_excel(Cf_Results, sheet_name='ANN128')
REPORT_PRES_ANN64.to_excel(Cf_Results, sheet_name='ANN64')
REPORT_PRES_ANN32.to_excel(Cf_Results, sheet_name='ANN32')

# Se guarda finalmente el archivo de excel con los resultados.
Cf_Results.save()

**GUARDADO DE LA SVM A TRAVÉS DE SCIKIT**

In [ ]:
# Se asocia una variable con el nombre del archivo de cada algoritmo.
filename1 = 'SVM.sav'
filename2 = 'LR.sav'
filename3 = 'KNN.sav'
filename4 = 'NBG.sav'
# Se ejecuta el guardado de los clasificadores. En este caso no se guardan las 
# redes neurales dado que no son usadas en la Raspberry.
pickle.dump(clf, open(filename1, 'wb'))
pickle.dump(grid_search_KNN, open(filename2, 'wb'))
pickle.dump(logreg_cv, open(filename3, 'wb'))
pickle.dump(clf_NB, open(filename4, 'wb'))
# Se asocia una variable con el nombre del archivo del escalizador.
scalerfile = 'scaler.sav'
# Se ejecuta el guardado del escalizador.
pickle.dump(transformer, open(scalerfile, 'wb'))